In [48]:
import sys
sys.path.append("../src")
import pandas as pd
from text_cleaning import clean_text
from nltk.tokenize import word_tokenize
from stylographic_features import get_stylographic_feat

from style_point_cloud import style_point_cloud

from distances import mod_hausdorff_dist

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_pickle("../data/csCL2000.pkl")

In [7]:
df.drop(["index", "pdf_link_no_export",
 "path_to_pdf", "pdf_metadata"], axis=1, inplace=True)

In [49]:
# Example of text cleaning
text1 = df.text_body[0]
text1 = clean_text(text1)

text2 = df.text_body[1]
text2 = clean_text(text2)

tokenized_text1 = word_tokenize(text1)
tokenized_text2 = word_tokenize(text2)
print(len(tokenized_text1), 
        len(tokenized_text2))

1732 4578


In [61]:
s_vec1 = style_point_cloud(text1, max_tokens = 1500,
 window_size=500, window_overlap=400,)
            #reduce="pca", redu_kwargs={"n_components": 4})
s_vec2 = style_point_cloud(text2,
 window_size=500, window_overlap=400, max_tokens = 2000, )
            #reduce="pca", redu_kwargs={"n_components": 4})

In [62]:
mod_hausdorff_dist(s_vec1, s_vec2)

68.12366756919658